# Library

In [398]:
import flodym as fd
from flodym import ExcelDimensionReader, DimensionDefinition
from flodym import ExcelParameterReader, ParameterDefinition

## System definition

In [399]:
dimension_definitions = [
    fd.DimensionDefinition(letter="t", name="time", dtype=int), 
    fd.DimensionDefinition(letter="c", name="time_construction", dtype=int), 
    fd.DimensionDefinition(letter="b", name="building sector", dtype=str), 
    fd.DimensionDefinition(letter="s", name="scenario", dtype=str), 
    fd.DimensionDefinition(letter="p", name="product", dtype=str), 
    fd.DimensionDefinition(letter="w", name="waste", dtype=str), 
]

In [400]:
parameter_definitions = [
    fd.ParameterDefinition(name="building area per capita", dim_letters=("t","b","s")),  
    fd.ParameterDefinition(name="population", dim_letters=("t",)), 
    fd.ParameterDefinition(name="concrete mass per area", dim_letters=("c","b","p","s")), 
    fd.ParameterDefinition(name="product share", dim_letters=("t","b","p")), 
    fd.ParameterDefinition(name="stock product age cohort", dim_letters=("t","c","s","b","p")),
    fd.ParameterDefinition(name="product lifetimes", dim_letters=("c","b","s")),
    fd.ParameterDefinition(name="waste share", dim_letters=("t", "w")),
]

In [401]:
process_names = [
    "sysenv", 
    "use", 
    "waste"   
]

In [402]:
flow_definitions = [
    fd.FlowDefinition(from_process_name="sysenv", to_process_name="use", dim_letters=("t", "s", "c", "b", "p")),
    fd.FlowDefinition(from_process_name="use", to_process_name="waste", dim_letters=("t", "w")),
]

In [403]:
stock_definitions = [
    fd.StockDefinition(
        name="use",
        process="use",
        dim_letters=("c", "s", "b", "p"),
        time_letter="c",
        subclass=fd.StockDrivenDSM,
        lifetime_model_class=fd.NormalLifetime,
    ),
        fd.StockDefinition(
        name="waste",
        process="waste",
        dim_letters=("t", "s", "w"),
        subclass=fd.SimpleFlowDrivenStock, # stock as the accumulation of inflow
    ),
]

In [404]:
mfa_definition = fd.MFADefinition(
    dimensions=dimension_definitions,
    parameters=parameter_definitions,
    processes=process_names,
    flows=flow_definitions,
    stocks=stock_definitions,
)

## Data import

In [405]:
dimension_file = "/Users/zhangyiwen/Documents/EMPA/Dds_school/Project/Data/dimension.xlsx"
dimension_files = {d.name: dimension_file for d in dimension_definitions}
dimension_sheets = {d.name: d.name for d in dimension_definitions}
reader = ExcelDimensionReader(
    dimension_files=dimension_files,
    dimension_sheets=dimension_sheets,
)

In [406]:
dims = reader.read_dimensions(dimension_definitions)

In [407]:
parameter_file = "/Users/zhangyiwen/Documents/EMPA/Dds_school/Project/Data/parameter.xlsx"
parameter_files = {p.name: parameter_file for p in parameter_definitions}
parameter_sheets = {p.name: p.name for p in parameter_definitions}
reader = ExcelParameterReader(
    parameter_files=parameter_files,
    parameter_sheets=parameter_sheets,
    allow_missing_values=True,
    allow_extra_values=True,
)

In [408]:
parameters = reader.read_parameters(parameter_definitions=parameter_definitions, dims=dims)

In [ ]:
class ConcreteMFA(fd.MFASystem):
    def compute(self): 
        print(f'dimensions population: {self.parameters["population"].dims}')
        print(f'dimensions building area per capita: {self.parameters["building area per capita"].dims}')

        future_building_stock = self.parameters["population"] * self.parameters["building area per capita"]
        print(f'dimensions self.stocks["use"].stock[...]: {self.stocks["use"].stock[...].dims}')
        print(self.stocks["use"].stock[...].sum_over(("b")))

        # use stock
        # provide the total stock and lifetime distribution data for inflow calculation

        self.stocks["use"].lifetime_model.set_prms(
            mean=self.parameters["product lifetimes"],
            std=0.3*self.parameters["product lifetimes"], # assume std = 30% of mean
        )
        survival_historic_values = self.stocks["use"].lifetime_model.sf
        c2 = fd.Dimension(name="time_construction2", letter="C", items=self.dims["c"].items)
        dims = fd.DimensionSet(dim_list=[c2] + self.stocks["use"].dims.dim_list)
        survival_historic = fd.FlodymArray(values=survival_historic_values, dims=dims)[{"C": 2015}]
        inflow_historic = self.parameters["stock product age cohort"][{"t": 2015}] / survival_historic
        
        dsm_historic = fd.InflowDrivenDSM(
            dims=inflow_historic.dims,
            lifetime_model=self.stocks["use"].lifetime_model,
            time_letter="c",
        )
        dsm_historic.inflow[...] = inflow_historic
        dsm_historic.compute()

        self.stocks["use"].stock[...] = dsm_historic.stock  
        # overwrite future development with prediction
        self.stocks["use"].stock[{"c": self.dims["t"]}] = future_building_stock * self.parameters["concrete mass per area"] * self.parameters["product share"]     
        # compute use stock inflow and outflow
        self.stocks["use"].compute()

        self.flows["sysenv => use"][...] = self.stocks["use"].inflow[{"c": self.dims["t"]}]

        
        # print(f'dimensions concrete mass per area: {self.parameters["concrete mass per area"].dims}')
        # print(f'dimensions product share: {self.parameters["product share"].dims}')
        
        # self.flows["sysenv => use"][...] = self.parameters["population"] * self.parameters["building area per capita"] * self.parameters["concrete mass per area"] * self.parameters["product share"]
        # print(f'dimensions flows["sysenv => use"] {self.flows["sysenv => use"][...].dims}')

        # print(self.flows["use"].inflow[...].sum_over(("c","b","p")))

        # print('---------------------------sysenv => use solved----------------------------')

        

        # print(f'dimensions self.stocks["use"] {self.stocks["use"].inflow[...].dims}')
        # print('INFLOW TO USE STOCK')
        # print(self.stocks["use"].inflow[...].sum_over(("c","b","p")))

        # # end-of-life flow
        # self.flows["use => waste"][...] = self.stocks["use"].outflow * self.parameters["waste share"]


In [410]:
ConcreteMFA_FR = ConcreteMFA.from_excel(
    definition=mfa_definition,
    dimension_files=dimension_files,
    parameter_files=parameter_files,
    dimension_sheets=dimension_sheets,
    parameter_sheets=parameter_sheets,
    allow_extra_parameter_values=True,
    allow_missing_parameter_values=True,
)

In [411]:
ConcreteMFA_FR.compute() #execute

dimensions population: DimensionSet (t) with shape (86,):
  't': 'time' with length 86
dimensions building area per capita: DimensionSet (t,b,s) with shape (86, 2, 2):
  't': 'time' with length 86
  'b': 'building sector' with length 2
  's': 'scenario' with length 2
dimensions self.stocks["use"].stock[...]: DimensionSet (c,s,b,p) with shape (201, 2, 2, 37):
  'c': 'time_construction' with length 201
  's': 'scenario' with length 2
  'b': 'building sector' with length 2
  'p': 'product' with length 37
FlodymArray 'use_stock' with dims (c,s,p) and shape (201, 2, 37);
Values:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


/Users/zhangyiwen/anaconda3/lib/python3.11/site-packages/flodym/flodym_arrays.py:349: RuntimeWarning: divide by zero encountered in divide
  1.0 / other.values,


In [412]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

ConcreteMFA_FR.check_mass_balance()

INFO:root:Checking mass balance of ConcreteMFA object...
INFO:root:Success - Mass balance is consistent!
